### Autoencoders and Neural Network for Place recognition with WiFi fingerprints
Implementation of algorithm discussed in <a href="https://arxiv.org/pdf/1611.02049v1.pdf">Low-effort place recognition with WiFi fingerprints using Deep Learning </a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import scale

In [2]:
dataset = pd.read_csv(".\\UJIndoorLoc\\trainingData.csv",header = 0)
features = scale(np.asarray(dataset.ix[:,0:520]))
labels = np.asarray(dataset["BUILDINGID"].map(str) + dataset["FLOOR"].map(str))
labels = np.asarray(pd.get_dummies(labels))

c:\python\python35\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  
c:\python\python35\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


#### Dividing UJIndoorLoc training data set into training and validation set

In [3]:
train_val_split = np.random.rand(len(features)) < 0.60
train_x = features[train_val_split]
train_y = labels[train_val_split]
val_x = features[~train_val_split]
val_y = labels[~train_val_split]

#### Using UJIndoorLoc validation data set as testing set

In [4]:
test_dataset = pd.read_csv(".\\UJIndoorLoc\\validationData.csv",header = 0)
test_features = scale(np.asarray(test_dataset.ix[:,0:520]))
test_labels = np.asarray(test_dataset["BUILDINGID"].map(str) + test_dataset["FLOOR"].map(str))
test_labels = np.asarray(pd.get_dummies(test_labels))

c:\python\python35\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  
c:\python\python35\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

In [6]:
n_input = 520 
n_hidden_1 = 256 
n_hidden_2 = 128 
n_hidden_3 = 64 

n_classes = labels.shape[1]

learning_rate = 0.01
training_epochs = 20
batch_size = 10

total_batches = dataset.shape[0] // batch_size

In [7]:
X = tf.placeholder(tf.float32, shape=[None,n_input])
Y = tf.placeholder(tf.float32,[None,n_classes])

# --------------------- Encoder Variables --------------- #

e_weights_h1 = weight_variable([n_input, n_hidden_1])
e_biases_h1 = bias_variable([n_hidden_1])

e_weights_h2 = weight_variable([n_hidden_1, n_hidden_2])
e_biases_h2 = bias_variable([n_hidden_2])

e_weights_h3 = weight_variable([n_hidden_2, n_hidden_3])
e_biases_h3 = bias_variable([n_hidden_3])

# --------------------- Decoder Variables --------------- #

d_weights_h1 = weight_variable([n_hidden_3, n_hidden_2])
d_biases_h1 = bias_variable([n_hidden_2])

d_weights_h2 = weight_variable([n_hidden_2, n_hidden_1])
d_biases_h2 = bias_variable([n_hidden_1])

d_weights_h3 = weight_variable([n_hidden_1, n_input])
d_biases_h3 = bias_variable([n_input])

# --------------------- DNN Variables ------------------ #

dnn_weights_h1 = weight_variable([n_hidden_3, n_hidden_2])
dnn_biases_h1 = bias_variable([n_hidden_2])

dnn_weights_h2 = weight_variable([n_hidden_2, n_hidden_2])
dnn_biases_h2 = bias_variable([n_hidden_2])

dnn_weights_out = weight_variable([n_hidden_2, n_classes])
dnn_biases_out = bias_variable([n_classes])

In [8]:
def encode(x):
    l1 = tf.nn.tanh(tf.add(tf.matmul(x,e_weights_h1),e_biases_h1))
    l2 = tf.nn.tanh(tf.add(tf.matmul(l1,e_weights_h2),e_biases_h2))
    l3 = tf.nn.tanh(tf.add(tf.matmul(l2,e_weights_h3),e_biases_h3))
    return l3
    
def decode(x):
    l1 = tf.nn.tanh(tf.add(tf.matmul(x,d_weights_h1),d_biases_h1))
    l2 = tf.nn.tanh(tf.add(tf.matmul(l1,d_weights_h2),d_biases_h2))
    l3 = tf.nn.tanh(tf.add(tf.matmul(l2,d_weights_h3),d_biases_h3))
    return l3

def dnn(x):
    l1 = tf.nn.relu(tf.add(tf.matmul(x,dnn_weights_h1),dnn_biases_h1))
    l2 = tf.nn.relu(tf.add(tf.matmul(l1,dnn_weights_h2),dnn_biases_h2))
    out = tf.nn.softmax(tf.add(tf.matmul(l2,dnn_weights_out),dnn_biases_out))
    return out

In [20]:
encoded = encode(X)
decoded = decode(encoded) 
y_ = dnn(encoded)

#### Choice of optimizer

In [10]:
us_cost_function = tf.reduce_mean(tf.pow(X - decoded, 2))
s_cost_function = -tf.reduce_sum(Y * tf.log(y_))
us_optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(us_cost_function)
s_optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(s_cost_function)

#### Check the accuracy
Following 2 lines is compare the results predicted by the network and the data of location in the testing set. `shape(y_) = [1111, 13]` while `shape(Y) = [1111]`as the there a 1111 testing data and the number of location lables is 13. The ocntents in `y_`, for each row, is the **possibility** that the point locates at the labled position. 
<br>
<br>
Function `tf.argmax(input, axis = 1)` will returns the **index** with the largest value across axes of a tensor but **not** the exact **value** of the maximum. If the shape of `input` is [12, 3], `axis = 1` will make the function takes index of maximum for each line, which means the shape of result will be [12].

In [11]:
correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#### Model architecture
Image take from: https://arxiv.org/pdf/1611.02049v1.pdf

<img src="AE.png">
<img src="NN.png">

In [24]:
with tf.Session() as session:
    tf.global_variables_initializer().run()
    
    # ------------ 1. Training Autoencoders - Unsupervised Learning ----------- #
    for epoch in range(training_epochs):
        epoch_costs = np.empty(0)
        for b in range(total_batches):
            offset = (b * batch_size) % (features.shape[0] - batch_size)
            batch_x = features[offset:(offset + batch_size), :]
            _, c = session.run([us_optimizer, us_cost_function],feed_dict={X: batch_x})
            epoch_costs = np.append(epoch_costs,c)
        print ("Epoch: ",epoch," Loss: ",np.mean(epoch_costs))
    print ("Unsupervised pre-training finished...")
    
    
    # ---------------- 2. Training NN - Supervised Learning ------------------ #
    for epoch in range(training_epochs):
        epoch_costs = np.empty(0)
        for b in range(total_batches):
            offset = (b * batch_size) % (train_x.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :]
            batch_y = train_y[offset:(offset + batch_size), :]
            _, c = session.run([s_optimizer, s_cost_function],feed_dict={X: batch_x, Y : batch_y})
            epoch_costs = np.append(epoch_costs,c)
        print ("Epoch: ",epoch," Loss: ",np.mean(epoch_costs)," Training Accuracy: ", \
            session.run(accuracy, feed_dict={X: train_x, Y: train_y}), \
            "Validation Accuracy:", session.run(accuracy, feed_dict={X: val_x, Y: val_y}))
            
    print ("Supervised training finished...")
    

    print ("\nTesting Accuracy:", session.run(accuracy, feed_dict={X: test_features, Y: test_labels}))
    
    print(session.run(y_, feed_dict={X: test_features, Y: test_labels}))

Epoch:  0  Loss:  0.943836372168
Epoch:  1  Loss:  0.86883025708
Epoch:  2  Loss:  0.832011357802
Epoch:  3  Loss:  0.810998824916
Epoch:  4  Loss:  0.797347557183
Epoch:  5  Loss:  0.787289342529
Epoch:  6  Loss:  0.779280373971
Epoch:  7  Loss:  0.772656919666
Epoch:  8  Loss:  0.767077457363
Epoch:  9  Loss:  0.762326091382
Epoch:  10  Loss:  0.758246766021
Epoch:  11  Loss:  0.754718664449
Epoch:  12  Loss:  0.751644609347
Epoch:  13  Loss:  0.748944953648
Epoch:  14  Loss:  0.746553831326
Epoch:  15  Loss:  0.744417540123
Epoch:  16  Loss:  0.742493021764
Epoch:  17  Loss:  0.740744806688
Epoch:  18  Loss:  0.739144690387
Epoch:  19  Loss:  0.737670312601
Unsupervised pre-training finished...
Epoch:  0  Loss:  5.11907597334  Training Accuracy:  0.473406 Validation Accuracy: 0.456804
Epoch:  1  Loss:  2.65517955388  Training Accuracy:  0.679792 Validation Accuracy: 0.67291
Epoch:  2  Loss:  1.53986906302  Training Accuracy:  0.78286 Validation Accuracy: 0.769368
Epoch:  3  Loss:  1

--------------------------------------------------------------------------------------------------------------------------